In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from datetime import datetime

import sqlalchemy
from sqlalchemy import create_engine, MetaData, PrimaryKeyConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date, Time

In [2]:
# Save csv files to variables
asteroid_csv = ["../Resources/cleaned_data/asteroid_impacts.csv", "../Resources/cleaned_data/asteroid_orbits.csv"]
stars_csv = "../Resources/cleaned_data/stars_classification.csv"
ufo_csv = "../Resources/cleaned_data/ufo_sightings.csv"

In [3]:
# Start your engines!
engine = create_engine("sqlite:///../../space_DB.sqlite")
conn = engine.connect()

In [4]:
# Create base
Base = declarative_base()

### Use SQLAlchemy to model table schema

In [5]:
# Asteroids
class impacts(Base):
    __tablename__ = "asteroid_impacts"
    
    id = Column(Integer, primary_key=True)
    object_name = Column(Text)
    period_start = Column(Integer)
    period_end = Column(Integer)
    possible_impacts = Column(Float)
    cumulative_impact_probability = Column(Float)
    asteroid_velocity = Column(Float)
    asteroid_magnitude = Column(Float)
    asteroid_diameter_km = Column(Float)
    cumulative_palermo_scale = Column(Float)
    maximum_palermo_scale = Column(Float)

In [6]:
class orbits(Base):
    __tablename__ = "asteroid_orbits"
    
    id = Column(Integer, primary_key=True)
    object_name = Column(Text)
    object_classification = Column(Text)
    epoch_TDB = Column(Integer)
    orbit_axis_AU = Column(Float)
    orbit_eccentricity = Column(Float)
    orbit_inclination_deg = Column(Float)
    perihelion_argument_deg = Column(Float)
    node_longitude_deg = Column(Float)
    mean_anomoly_deg = Column(Float)
    perihelion_distance_AU = Column(Float)
    aphelion_distance_AU = Column(Float)
    orbital_period_yr = Column(Float)
    minimum_orbit_intersection_distance_AU = Column(Float)
    orbital_reference = Column(Float)
    asteroid_magnitude = Column(Float)

In [7]:
# Stars
class stars(Base):
    __tablename__ = "star_classification"
    
    id = Column(Integer, primary_key=True)
    Temperature = Column(Integer)
    L = Column(Float)
    R = Column(Float)
    A_M = Column(Float)
    Color = Column(Text)
    Spectral_Class = Column(Text)
    Type = Column(Integer)

In [8]:
# UFO
class ufo(Base):
    __tablename__ = "ufo_sightings"
    
    id = Column(Integer, primary_key=True)
    date_occured = Column(Text)
    time_occured = Column(Text)
    city = Column(Text)
    state = Column(Text)
    country = Column(Text)
    shape = Column(Text)
    duration_seconds = Column(Integer)
    duration_hours_min = Column(Text)
    comments = Column(Text)
    date_posted = Column(Text)
    latitude = Column(Integer)
    longitude = Column(Integer)

When just trying type as Date or Time (or datetime, timestamp, sql_variant) get:

These codes worked in jupyter notebook
 
class ufo(Base):

     __tablename__ = "ufo_sightings"
    
     id = Column(Integer, primary_key=True)
     date_occured = Column(sqlalchemy.Date)
     time_occured = Column(sqlalchemy.Time)

but get 
StatementError: (builtins.TypeError) SQLite Date type only accepts Python date objects as input.
 when attempting to put in sqlite db
 
 ######################################################################
 
"SQLite does not have a storage class set aside for storing dates and/or times. Instead, the built-in Date And Time Functions of SQLite are capable of storing dates and times as TEXT, REAL, or INTEGER values:

TEXT as ISO8601 strings ("YYYY-MM-DD HH:MM:SS.SSS").
REAL as Julian day numbers, the number of days since noon in Greenwich on November 24, 4714 B.C. according to the proleptic Gregorian calendar.
INTEGER as Unix Time, the number of seconds since 1970-01-01 00:00:00 UTC"

Source=
https://www.w3resource.com/sqlite/sqlite-data-types.php

In [9]:
# create all tables reflect to bind
Base.metadata.create_all(engine)
metadata = MetaData(bind=engine)
metadata.reflect()

### Use Pandas to read csv into a list of row objects

In [10]:
df = pd.read_csv(asteroid_csv[0], dtype=object)
impacts_data = df.to_dict(orient='records')

df = pd.read_csv(asteroid_csv[1], dtype=object)
orbits_data = df.to_dict(orient='records')

df = pd.read_csv(stars_csv, dtype=object)
stars_data = df.to_dict(orient='records')

df = pd.read_csv(ufo_csv, dtype=object)
ufo_data = df.to_dict(orient='records')

### Insert data into table using SQLAlchemy

In [11]:
impacts_table = sqlalchemy.Table('asteroid_impacts', metadata, PrimaryKeyConstraint('id'),
                            autoload=True, extend_existing=True)

orbits_table = sqlalchemy.Table('asteroid_orbits', metadata, PrimaryKeyConstraint('id'),
                            autoload=True, extend_existing=True)

stars_table = sqlalchemy.Table('star_classification', metadata, PrimaryKeyConstraint('id'),
                            autoload=True, extend_existing=True)

ufo_table = sqlalchemy.Table('ufo_sightings', metadata, PrimaryKeyConstraint('id'),
                            autoload=True, extend_existing=True)

### Execute (old) delete() and  execute (new) insert ()

In [12]:
conn.execute(impacts_table.delete())
conn.execute(impacts_table.insert(), impacts_data)

In [13]:
conn.execute(orbits_table.delete())
conn.execute(orbits_table.insert(), orbits_data)

In [14]:
conn.execute(stars_table.delete())
conn.execute(stars_table.insert(), stars_data)

In [15]:
conn.execute(ufo_table.delete())
conn.execute(ufo_table.insert(), ufo_data)